<a href="https://colab.research.google.com/github/XescLlabres/tfgFLM/blob/main/WDBC/TaulaLOOCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.model_selection import LeaveOneOut

In [ ]:
data1 = pd.read_csv('data.csv.xls')

# New Section

In [ ]:

np.shape(data1)

(569, 33)

In [ ]:
data1.tail()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN
568,92751,B,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,...,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039,NaN


In [ ]:
data1.drop(['Unnamed: 32','id'],axis=1,inplace=True)
data1.diagnosis=[1 if each=="M" else 0 for each in data1.diagnosis]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut

# Evitam espais als noms de les columnes
data1.columns = data1.columns.str.strip()

# Seleccionam columna a predir
y = data1.loc[:, "diagnosis"].values

# Llevam columna a predir i columna de identificació
X = data1.loc[:, (data1.columns != "diagnosis") & (data1.columns != "id")].values

# Escalador de característiques

scaler = StandardScaler()

# Escalar X

X_scaled = scaler.fit_transform(X)


# Crear el model i el LOOCV
loo = LeaveOneOut()
model = LogisticRegression(max_iter=1000)  # Feim gran el max_iter per donar temps al solver

# Feim llista per a guardar-hi les probabilitats
all_bootstrap_probabilities = []

# Aplicam LOOCV amb Bootstrap
for train_index, test_index in loo.split(X_scaled):  # Usam l'X escalat
    # Separam train i test
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Guardam les probabilitats resultants del pacient que deixam fora amb el bootstrap
    bootstrapped_probabilities = []

    # Generam les 1000 mostres de bootstrap i feim les prediccions
    for _ in range(1000):
        # Generar mostra de bootstrap
        X_train_bootstrap, y_train_bootstrap = resample(X_train, y_train, replace=True)

        # Entrenar el model amb els pacients seleccionats
        model.fit(X_train_bootstrap, y_train_bootstrap)

        # Predir la probabilitat de que diagnosis sigui 1 per el pacient restant
        prob = model.predict_proba(X_test)[:, 1][0]
        bootstrapped_probabilities.append(prob) #Afegir probabilitat una a una

    # Guardar a la llista i seguir amb el pròxim pacient
    all_bootstrap_probabilities.append(bootstrapped_probabilities)

# Convertir la llista de probabilitats a un DataFrame
df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

# Guardar el DataFrame en un CSV
df_probabilities.to_csv("bootstrap_probabilities.csv", index=False)



In [ ]:
#Afegir identificador a la llista de probabilitats per a posteriors anàlisis


df_probabilities = pd.DataFrame(all_bootstrap_probabilities)
data1 = pd.read_csv('data.csv.xls')
data1.drop(['Unnamed: 32'],axis=1,inplace=True)
df_probabilities.insert(0, 'id', data1['id'].values)
df_probabilities.to_csv("bootstrap_probabilities_id.csv", index=False)



print("Archiu guardat amb id + probabilitats.")


Archiu guardat amb id + probabilitats.
